In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4`
import $ivy.`sh.almond::almond-spark:0.6.0`
import $ivy.`io.projectglow::glow:0.2.0`
import $ivy.`com.github.pathikrit::better-files:3.8.0`
import io.projectglow.Glow
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import better.files._
import File._
import java.io.{File => JFile}

import $ivy.$                                  

import $ivy.$                              

import $ivy.$                           

import io.projectglow.Glow

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

In [2]:
val ss = {
  NotebookSparkSession
    .builder()
    .config("spark.sql.shuffle.partitions", "1")
    .config("spark.ui.enabled", "false")
    .config("spark.driver.host", "localhost")
    .master("local[*]")
    .getOrCreate()
}
import ss.implicits._
Glow.register(ss)

Loading spark-stubs


Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.jar.sha1
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0-sources.jar
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo

Creating SparkSession


20:02:13.310 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


ss: SparkSession = org.apache.spark.sql.SparkSession@2467e742
import ss.implicits._


In [10]:
val data_dir = home / "data" / "gwas" / "tutorial" / "1_QC_GWAS"

data_dir: File = /home/eczech/data/gwas/tutorial/1_QC_GWAS

## Load and Cache

In [11]:
val df = ss.read.format("plink").load(data_dir / "HapMap_3_r3_1.bed" toString)

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [12]:
df.printSchema

root
 |-- contigName: string (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- position: double (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genotypes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- sampleId: string (nullable = true)
 |    |    |-- calls: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [13]:
df.write.format("parquet").save(data_dir / "HapMap_3_r3_1.parquet" toString)

save at cmd12.sc:1

15 / 15

## Reload

In [14]:
val df = ss.read.parquet(data_dir / "HapMap_3_r3_1.parquet" toString)

parquet at cmd13.sc:1

1 / 1

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [15]:
df.count

count at cmd14.sc:1

15 / 15

count at cmd14.sc:1

1 / 1

res14: Long = 1457897L

In [16]:
df.groupBy(size($"genotypes.calls")).count.show

show at cmd15.sc:1

15 / 15

show at cmd15.sc:1

1 / 1

+---------------------+-------+
|size(genotypes.calls)|  count|
+---------------------+-------+
|                  165|1457897|
+---------------------+-------+



In [16]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .show(3, false)

show at cmd15.sc:3

1 / 1

+----------+-----------+--------+------+------+---------------+----------------+----------------------+
|contigName|names      |position|start |end   |referenceAllele|alternateAlleles|genotypes             |
+----------+-----------+--------+------+------+---------------+----------------+----------------------+
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1328_NA06989, [0, 0]]|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1377_NA11891, [0, 0]]|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1349_NA11843, [0, 0]]|
+----------+-----------+--------+------+------+---------------+----------------+----------------------+
only showing top 3 rows



In [18]:
// Show number of calls per sample + variant combo
// Should be across num variants * num samples data points
print(s"Expected: ${1457897 * 165}")
df
    .withColumn("genotypes", explode($"genotypes"))
    .groupBy(size($"genotypes.calls")).count.show

Expected: 240553005

show at cmd17.sc:4

15 / 15

show at cmd17.sc:4

1 / 1

+---------------------+---------+
|size(genotypes.calls)|    count|
+---------------------+---------+
|                    2|240553005|
+---------------------+---------+



In [19]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .groupBy(size($"names"), size($"alternateAlleles")).count.show

show at cmd18.sc:3

15 / 15

show at cmd18.sc:3

1 / 1

+-----------+----------------------+---------+
|size(names)|size(alternateAlleles)|    count|
+-----------+----------------------+---------+
|          1|                     1|240553005|
+-----------+----------------------+---------+



#### Call Rate

Calculate variant call rates (i.e. across samples) manually and compare to ```call_summary_stats```.

In [20]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .withColumn("call", explode_outer($"genotypes.calls"))
    .withColumn("sampleId", $"genotypes.sampleId")
    .drop("genotypes")
    .show(3, false)

show at cmd19.sc:6

1 / 1

+----------+-----------+--------+---------+---------+---------------+----------------+----+------------+
|contigName|names      |position|start    |end      |referenceAllele|alternateAlleles|call|sampleId    |
+----------+-----------+--------+---------+---------+---------------+----------------+----+------------+
|6         |[rs1890312]|0.0     |153585564|153585565|G              |[A]             |0   |1328_NA06989|
|6         |[rs1890312]|0.0     |153585564|153585565|G              |[A]             |0   |1328_NA06989|
|6         |[rs1890312]|0.0     |153585564|153585565|G              |[A]             |0   |1377_NA11891|
+----------+-----------+--------+---------+---------+---------------+----------------+----+------------+
only showing top 3 rows



In [38]:
// See https://glow.readthedocs.io/en/latest/etl/utility-functions.html
def dfc = df
    .selectExpr("*", "genotype_states(genotypes) as states")
    .withColumn("variantId", $"names"(0))
    .withColumn("sampleId", $"genotypes.sampleId")
    .selectExpr(
        "variantId", 
        "explode(arrays_zip(sampleId, states)) as genotypes"
    )
    .select("variantId", "genotypes.*")
dfc.printSchema

root
 |-- variantId: string (nullable = true)
 |-- sampleId: string (nullable = true)
 |-- states: integer (nullable = true)



defined function dfc

In [40]:
def dfcv = dfc
    .groupBy("variantId").agg(mean(($"states" >= 0).cast("float")).as("callRate"))
dfcv.show(3)

show at cmd39.sc:3

15 / 15

show at cmd39.sc:3

1 / 1

+----------+--------+
| variantId|callRate|
+----------+--------+
| rs1890312|     1.0|
|  rs317105|     1.0|
|rs10499272|     1.0|
+----------+--------+
only showing top 3 rows



defined function dfcv

In [43]:
def dfcg = df
    .selectExpr(
        "element_at(names, 1) as variantId", 
        "call_summary_stats(genotypes).callRate as callRate"
    )
dfcg.show(3) 

show at cmd42.sc:6

1 / 1

+----------+--------+
| variantId|callRate|
+----------+--------+
| rs1890312|     1.0|
|  rs317105|     1.0|
|rs10499272|     1.0|
+----------+--------+
only showing top 3 rows



defined function dfcg

In [48]:
def dfj = dfcv
    .withColumnRenamed("callRate", "callRateManual")
    .join(dfcg.withColumnRenamed("callRate", "callRateFunction"), Seq("variantId"), "outer")
dfj.printSchema

root
 |-- variantId: string (nullable = true)
 |-- callRateManual: double (nullable = true)
 |-- callRateFunction: double (nullable = true)



defined function dfj

In [49]:
dfj.groupBy($"callRateManual" === $"callRateFunction").count.show

show at cmd48.sc:1

15 / 15

show at cmd48.sc:1

15 / 15

show at cmd48.sc:1

1 / 1

show at cmd48.sc:1

1 / 1

+-----------------------------------+-------+
|(callRateManual = callRateFunction)|  count|
+-----------------------------------+-------+
|                               true|1457897|
+-----------------------------------+-------+

